In [2]:
import os
os.environ['KERAS_BACKEND']="jax"

import numpy as np
from pathlib import Path
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import keras
import pickle

class MyDNN:
    def __init__(self,tag,model,scalerX=None,scalerY=None) -> None:
        self.scalerX=scalerX
        self.scalerY=scalerY
        self.model=model
        self.tag=tag

    def save(self):
        self.model.save(f"./result/DNN/{self.tag}_model.keras")
        scaler=[self.scalerX,self.scalerY]
        with open(f"./result/DNN/{self.tag}_scaler.pkl",'wb') as fout:
            pickle.dump(scaler,fout)

    @staticmethod
    def load(tag):
        model=keras.saving.load_model(f"./result/DNN/{tag}_model.keras")

        with open(f"./result/DNN/{tag}_scaler.pkl",'rb') as fin:
            scaler=pickle.load(fin)

        return MyDNN(tag=tag,model=model,scalerX=scaler[0],scalerY=scaler[1])

    def predict(self,x):
        if self.scalerX:
            postX=self.scalerX.transform(x)
        else:
            postX=x
        predY=self.model.predict(postX,verbose=0)
        if self.scalerY:
            Y=self.scalerY.inverse_transform(predY)
        else:
            Y=predY
        return Y

In [98]:
cfd_dir = Path("./result/CFD/")
cfdData = []
for i in range(1500):
    filename = cfd_dir / f"{i:03d}.csv"
    tmp = np.loadtxt(fname=filename, delimiter=",", ndmin=2)[:, [0, 1, 3]]
    if np.any(tmp[:, [0, 1]] < 0.0):
        print(i)
    tmp = tmp.ravel()
    if len(tmp) != 9:
        print(f"{i:03d} is error!")
    else:
        cfdData.append(tmp)
cfdData = np.vstack(cfdData)
cadData = np.loadtxt("./sample.csv", delimiter=",", ndmin=2)
print(cfdData.shape, cadData.shape)

# flag=600
# trainX=cadData[:flag,:]
# testX=cadData[flag:,:]
# trainY=cfdData[:flag,:]
# testY=cfdData[flag:,:]
cfdID = 8
X_train, X_test, Y_train, Y_test = train_test_split(
    cadData, cfdData[:, [cfdID]], test_size=75, shuffle=False, random_state=1
)
print(X_train.shape, Y_train.shape)

Scaler=MinMaxScaler

scalerX = Scaler()
scalerY = Scaler()

scalerX = scalerX.fit(X_train)
scalerY = scalerY.fit(Y_train)

X_train = scalerX.transform(X_train)
X_test = scalerX.transform(X_test)

Y_train = scalerY.transform(Y_train)
Y_test = scalerY.transform(Y_test)

(1500, 9) (1500, 25)
(1425, 25) (1425, 1)


In [99]:
keras.utils.set_random_seed(0)

batch_size = 64
epochs = 500
width=100
n_hidden_layer=20


hidden_layers=[]
for _ in range(n_hidden_layer):
    hidden_layers.append(keras.layers.Dense(units=width))
    hidden_layers.append(keras.layers.ELU())

model = keras.Sequential(layers=[
    keras.layers.Input(shape=(25,), name="input"),
    *hidden_layers,
    # keras.layers.Dropout(rate=0.5),
    keras.layers.Dense(units=1, name="output")
])
# model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=keras.losses.Huber(delta=0.1),
    metrics=[keras.metrics.R2Score()],
)

callbacks = [
    # keras.callbacks.ModelCheckpoint(
    #     filepath="./keras_data/model_{epoch:03d}-{val_loss:.2f}.keras", save_best_only=True
    # ),
    # keras.callbacks.EarlyStopping(monitor="val_r2_score", patience=100),
]

hist: keras.callbacks.History = model.fit(
    X_train,
    Y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0,
    callbacks=callbacks,
    verbose=1,
)

Epoch 1/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 8s 247ms/step - loss: 0.0406 - r2_score: -27.1052
Epoch 2/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0039 - r2_score: -0.0625
Epoch 3/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0030 - r2_score: 0.1648
Epoch 4/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0021 - r2_score: 0.3244  
Epoch 5/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0021 - r2_score: 0.3494
Epoch 6/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0016 - r2_score: 0.4207
Epoch 7/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0019 - r2_score: 0.3455  
Epoch 8/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0019 - r2_score: 0.3552
Epoch 9/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0022 - r2_score: 0.2896
Epoch 10/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0019 - r2_score: 0.2981
Epoch 11/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0020 - r2_score: 0.3333
Epoch 12/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - 

In [100]:
# model=keras.saving.load_model("./keras_data/model_115-0.01.keras")
score = model.evaluate(X_test, Y_test, verbose=1)
IDs=[0,799,1460,1462,1464,1466,1470,1475,1480]
cadData_nor=scalerX.transform(cadData)
predY=model.predict(cadData_nor[IDs,:],verbose=False)
print(scalerY.inverse_transform(predY).ravel())
print(cfdData[[IDs],[cfdID]].ravel())

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 379ms/step - loss: 0.0010 - r2_score: 0.6758
[0.44051388 0.42408672 0.4675504  0.39254293 0.44640553 0.42627788
 0.45213532 0.46582368 0.44430625]
[0.440001 0.42152  0.47043  0.392378 0.448794 0.409575 0.444676 0.463959
 0.442496]


In [101]:
mydnn=MyDNN(tag=cfdID,model=model,scalerX=scalerX,scalerY=scalerY)
mydnn.save()

mydnn=MyDNN.load(tag=cfdID)
predY=mydnn.predict(cadData[IDs,:])
print(predY.ravel())

[0.44051388 0.42408672 0.4675504  0.39254293 0.44640553 0.42627788
 0.45213532 0.46582368 0.44430625]
